In [1]:
import numpy as np
import matplotlib.pyplot as plt
import openmc
import pandas as pd

# Data

In [23]:
# data
# costs
c_yc = 60           # $ / lb of yellow cake
c_con = 11.5        # $ / kg U
c_swu = 110         # $ / SWU
c_ft = 230          # $ / kg U to fabricate and transport

# enrichments
x_nu = 0.711 / 100  # enrichment of natural U
x_en = 4.2 / 100    # enrichment of fuel
x_tl = 0.25 / 100   # enrichments of tails

# losses
l_con = 0.6 / 100   # conversion loss
l_fab = 0.7 / 100   # fabrication loss


# formatting data
m_o = openmc.data.atomic_weight('O')
m_u = openmc.data.atomic_weight('U')

m_yc = (3 * m_u) + (8 * m_o)
m_ratio = (3 * m_u) / m_yc

lb_per_kg = 2.2046
c_yc *= lb_per_kg / m_ratio  # ($ per lb_yc) * (lb per kg) / (U per yc)

# Functions

In [28]:
def sep_pot(x):
    """returns seperation potential for an enrichment x"""
    return (2 * x - 1) * np.log(x / (1 - x))


def sep_factor(xf=x_nu, xp=x_en, xw=x_tl):
    """find the cost to enrich m [kg] of UF6"""
    fop = (xp - xw) / (xf - xw)
    wop = (xp - xf) / (xf - xw)

    vp = sep_pot(xp)
    vf = sep_pot(xf)
    vw = sep_pot(xw)

    return vp + (wop * vw) - (fop * vf)


# functions
def fuel_fab(xp=x_en, do_print=False):
    """calculates fuel fabrication cost using Eq 4.1"""
    # calcs
    sf = sep_factor(xp=xp)
    fop = (xp - x_tl) / (x_nu - x_tl)

    # costs
    c1 = ((c_yc / (1 - l_con) / (1 - l_fab)) + (c_con / (1 - l_fab))) * fop
    c2 = (c_swu / (1 - l_fab)) * sf
    c3 = c_ft

    if do_print: print(f"{c1}\n{c2}\n{c3}")

    return c1 + c2 + c3

# Question 2

In [25]:
cost = fuel_fab(xp=x_en)
print(f"FF w/ 4.2%: ${round(cost, 2)}/kg")

FF w/ 4.2%: $2375.23/kg


# Question 3

In [26]:
c1 = fuel_fab(xp=x_en)
c2 = fuel_fab(xp=(4.6 / 100))

diff = (c2 - c1) / c1 * 100

print(f"FF w/ 4.2%: ${round(c1, 2)}/kg")
print(f"FF w/ 4.6%: ${round(c2, 2)}/kg")
print(f"\nPercent Diff: {round(diff, 5)}%")

FF w/ 4.2%: $2375.23/kg
FF w/ 4.6%: $2614.83/kg

Percent Diff: 10.08785%
